In [1]:
import random
from datetime import date
import numpy as np
import pandas as pd
from ortools.sat.python import cp_model

In [2]:
# Constants
DAYS = 7
WEEKDAYS_DICTIONARY = {0 : "Sun", 1 : "Mon", 2 : "Tue", 3 : "Wed", \
                       4 : "Thu", 5 : "Fri", 6 : "Sat"}

PART_TIME_SHIFTS_AT_MOST = 3
PART_TIME_SHIFTS_AT_LEAST = 2
FULL_TIME_SHIFTS_AT_MOST = 6
FULL_TIME_SHIFTS_AT_LEAST = 4
EXCEL_NAME = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSa76sKQuHwLpRbJhkoZmAspIgf0M0BaLHgbVeem5LFy9G6NetJ4Oro1rN_SsjPL3Qra3Tung7larlG/pub?output=xlsx"
availability = pd.read_excel(EXCEL_NAME, "Availability")
requirements = pd.read_excel(EXCEL_NAME, "Shifts Requirement")

In [7]:
def bartender_schedule():
    SHIFTS = 2
    SHIFTS_DICTIONARY = {0 : "AM", 1 : "PM"}
    # Read the Excel
    availability = pd.read_excel(EXCEL_NAME, "Availability")
    employees = availability.iloc[4, 22]
    part_time = availability.iloc[2:, 1:3].reset_index(drop=True)
    preferences = availability.iloc[2:, 3:17].reset_index(drop=True)
    number_employee_shifts = requirements.iloc[3:5, 2:].reset_index(drop=True)
    employees_name = availability.iloc[2:availability.iloc[4, 22]+2, 1].reset_index(drop=True)
        
    # Make a dictionary that every employee's index is the key, value is 0 or 1, 0 means not part time, 1 is part time
    is_part_time = {}
    for e in range(employees):
        if part_time.iat[e, 1] == "Part-Time":
            is_part_time[e] = 1
        else:
            is_part_time[e] = 0

    # Make preference matrix
    prefer_value = [[[0 for s in range(SHIFTS)] for d in range(DAYS)] for e in range(employees)]

    for e in range(employees):
        count = 0
        while count < (DAYS * SHIFTS):
            d = count // SHIFTS
            s = count % SHIFTS
            if preferences.iat[e, count] == "Available":
                prefer_value[e][d][s] = 1
            elif preferences.iat[e, count] == "Not Preferred":
                prefer_value[e][d][s] = 0
            else:
                prefer_value[e][d][s] = -50
            count += 1

    # Each shift require different number of employees, so below is requirement matrix
    shifts_requirement = [[0 for s in range(SHIFTS)] for d in range(DAYS)]
    for d in range(DAYS):
        for s in range(SHIFTS):
            shifts_requirement[d][s] = int(number_employee_shifts.iat[s, d])

    # create instance of cp_model
    model = cp_model.CpModel()

    # Assign all the shifts to x[e,d,s] e means employee, d means days, s means morning shift or evening shift
    x = {}
    for e in range(employees):
        for d in range(DAYS):
            for s in range(SHIFTS):
                x[(e, d, s)] = model.NewBoolVar("x[%d,%d,%d]" % (e, d, s))

    # Constraint 1: Each shift in a week require different number of employee
    for d in range(DAYS):
        for s in range(SHIFTS):
            model.Add(sum(x[e, d, s] for e in range(employees)) == shifts_requirement[d][s])

    # Constraint 2: No one can work two shifts in a day
    for e in range(employees):
        for d in range(DAYS):
            model.Add(sum(x[e, d, s] for s in range(SHIFTS)) <= 1)

    # Constraint 3: No one can work two consecutive shifts(avoiding today's evening shift and tomorrow's morning shift)
    for d in range(DAYS):
        for e in range(employees):
            for s in range(SHIFTS):
                model.Add((x[e, d, 1] + x[e, (d+1)% DAYS, 0]) <= 1)

    # Constraint 4: Part time employee and full time employee have different number of shifts in a week
    for e in range(employees):
        if is_part_time[e] == 1:
            model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) <= PART_TIME_SHIFTS_AT_MOST)
            model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) >= PART_TIME_SHIFTS_AT_LEAST)
        else:
            model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) <= FULL_TIME_SHIFTS_AT_MOST)
            model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) >= FULL_TIME_SHIFTS_AT_LEAST)

    # Constraint 5: Each employee at least have one shift at friday or saturday evening unless she/he don't want to
    for e in range(employees):
        if prefer_value[e][5][1] != -50 and prefer_value[e][6][1] != -50:
            model.Add((x[e, 5, 1] + x[e, 6, 1]) >= 1)

    # Constraint 6: George only work at friday and saturday night.
    for e in range(employees):
        if employees_name[e] == "George":
            model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) == 2)
            model.Add((x[e, 5, 1] + x[e, 6, 1]) == 2)

    # Objective Function find total Maximum Happy points
    model.Maximize(sum(prefer_value[e][d][s] * x[e,d,s] for e in range(employees) \
                       for d in range(DAYS) for s in range(SHIFTS)))

    # Call Solver
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Call Solver
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Output schedule to excel
    schedule = {"%s %s" % (WEEKDAYS_DICTIONARY[d], SHIFTS_DICTIONARY[s]) : [] for d in range(DAYS) for s in range(SHIFTS)}
    if status == cp_model.OPTIMAL:
        for e in range(employees):
            for d in range(DAYS):
                for s in range(SHIFTS):
                    if solver.Value(x[e, d, s]) == 1 and prefer_value[e][d][s] == -50:
                        schedule[f"{WEEKDAYS_DICTIONARY[d]} {SHIFTS_DICTIONARY[s]}"].append(employees_name[e] + "(X)")
                    elif solver.Value(x[e, d, s]) == 1:
                        schedule[f"{WEEKDAYS_DICTIONARY[d]} {SHIFTS_DICTIONARY[s]}"].append(employees_name[e])
                    else:
                        schedule[f"{WEEKDAYS_DICTIONARY[d]} {SHIFTS_DICTIONARY[s]}"].append("-")
        employees_name = employees_name.to_frame(name="Employees' Name")
        df = pd.DataFrame(schedule, index=employees_name["Employees' Name"])
        return df
    else:
        print("No optimal schedule found")

In [6]:
def services_schedule():
    SHIFTS = 3
    SHIFTS_DICTIONARY = {0 : "AM", 1 : "MID", 2 : "PM"}
    # Read the excel
    employees = availability.iloc[5, 22]
    part_time = availability.iloc[22:, 2].reset_index(drop=True)
    preferences = availability.iloc[22:, 3:24].reset_index(drop=True)
    number_employee_shifts = requirements.iloc[8:, 2:].reset_index(drop=True)
    employees_name = availability.iloc[22:, 1].reset_index(drop=True)

    # Make a dictionary that every employee's index is the key, value is 0 or 1, 0 means not part time, 1 is part time
    is_part_time = {}
    for e in range(employees):
        if part_time.iat[e] == "Part-Time":
            is_part_time[e] = 1
        else:
            is_part_time[e] = 0

    # Make preference matrix
    prefer_value = [[[0 for s in range(SHIFTS)] for d in range(DAYS)] for e in range(employees)]

    for e in range(employees):
        count = 0
        while count < (DAYS * SHIFTS):
            d = count // SHIFTS
            s = count % SHIFTS
            if preferences.iat[e, count] == "Available":
                prefer_value[e][d][s] = 1
            elif preferences.iat[e, count] == "Not Preferred":
                prefer_value[e][d][s] = 0
            else:
                prefer_value[e][d][s] = -50
            count += 1

    # Each shift require different number of employees, so below is requirement matrix
    shifts_requirement = [[0 for s in range(SHIFTS)] for d in range(DAYS)]
    for d in range(DAYS):
        for s in range(SHIFTS):
            shifts_requirement[d][s] = int(number_employee_shifts.iat[s, d])

    # create instance of cp_model
    model = cp_model.CpModel()

    # Assign all the shifts to x[e,d,s] e means employee, d means days, s means morning shift or evening shift
    x = {}
    for e in range(employees):
        for d in range(DAYS):
            for s in range(SHIFTS):
                x[(e, d, s)] = model.NewBoolVar("x[%d,%d,%d]" % (e, d, s))

    # Constraint 1: Each shift in a week require different number of employee
    for d in range(DAYS):
        for s in range(SHIFTS):
            model.Add(sum(x[e, d, s] for e in range(employees)) == shifts_requirement[d][s])

    # Constraint 2: No one can work two shifts in a day
    for e in range(employees):
        for d in range(DAYS):
            model.Add(sum(x[e, d, s] for s in range(SHIFTS)) <= 1)

    # Constraint 3: No one can work two consecutive shifts(avoiding today's evening shift and tomorrow's morning shift)
    for d in range(DAYS):
        for e in range(employees):
            for s in range(SHIFTS):
                model.Add((x[e, d, 2] + x[e, (d+1) % DAYS, 0]) <= 1)

    # Constraint 4: Part time employee and full time employee have different number of shifts in a week
    for e in range(employees):
        if is_part_time[e] == 1:
            model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) <= PART_TIME_SHIFTS_AT_MOST)
            model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) >= PART_TIME_SHIFTS_AT_LEAST)
        else:
            model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) <= FULL_TIME_SHIFTS_AT_MOST)
            model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) >= FULL_TIME_SHIFTS_AT_LEAST)

    # Constraint 5: Each employee at least have one shift at friday or saturday evening unless she/he don't want to
    # for e in range(employees):
    #    if prefer_value[e][5][2] != -50 and prefer_value[e][6][2] != -50:
    #        model.Add((x[e, 5, 2] + x[e, 6, 2]) >= 1)

    # Objective Function find total Maximum Happy points
    model.Maximize(sum(prefer_value[e][d][s] * x[e,d,s] for e in range(employees) \
                       for d in range(DAYS) for s in range(SHIFTS)))

    # Call Solver
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Output schedule to excel
    schedule = {"%s %s" % (WEEKDAYS_DICTIONARY[d], SHIFTS_DICTIONARY[s]) : [] for d in range(DAYS) for s in range(SHIFTS)}
    if status == cp_model.OPTIMAL:
        for e in range(employees):
            for d in range(DAYS):
                for s in range(SHIFTS):
                    if solver.Value(x[e, d, s]) == 1 and prefer_value[e][d][s] == -50:
                        schedule[f"{WEEKDAYS_DICTIONARY[d]} {SHIFTS_DICTIONARY[s]}"].append(employees_name[e] + "(X)")
                    elif solver.Value(x[e, d, s]) == 1:
                        schedule[f"{WEEKDAYS_DICTIONARY[d]} {SHIFTS_DICTIONARY[s]}"].append(employees_name[e])
                    else:
                        schedule[f"{WEEKDAYS_DICTIONARY[d]} {SHIFTS_DICTIONARY[s]}"].append("-")
        employees_name = employees_name.to_frame(name="Employees' Name")
        df = pd.DataFrame(schedule, index=employees_name["Employees' Name"])
        return df
    else:
        print("No optimal schedule found")

In [9]:
bartender_df = bartender_schedule()
service_df = services_schedule()
writer = pd.ExcelWriter("./TapHouse services and bartenders schedule.xlsx", engine = "openpyxl")

bartender_df.to_excel(writer, sheet_name = "bartenders schedule")
service_df.to_excel(writer, sheet_name = "services schedule")
writer.save()